# Neeps- Medium

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app15-2")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@6ce4ed35

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@308745e5

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val ut_staff = hiveCxt.table("sqlzoo.ut_staff")
val ut_student = hiveCxt.table("sqlzoo.ut_student")
val ut_event = hiveCxt.table("sqlzoo.ut_event")
val ut_room = hiveCxt.table("sqlzoo.ut_room")
val ut_attends = hiveCxt.table("sqlzoo.ut_attends")
val ut_teaches = hiveCxt.table("sqlzoo.ut_teaches")
val ut_occurs = hiveCxt.table("sqlzoo.ut_occurs")
val ut_modle = hiveCxt.table("sqlzoo.ut_modle")
val ut_week = hiveCxt.table("sqlzoo.ut_week")

ut_staff: DataFrame = [id: string, name: string]
ut_student: DataFrame = [id: string, name: string ... 2 more fields]
ut_event: DataFrame = [id: string, modle: string ... 5 more fields]
ut_room: DataFrame = [id: string, name: string ... 2 more fields]
ut_attends: DataFrame = [student: string, event: string]
ut_teaches: DataFrame = [staff: string, event: string]
ut_occurs: DataFrame = [event: string, week: string]
ut_modle: DataFrame = [id: string, name: string]
ut_week: DataFrame = [id: string, wkstart: string]

## 6.
**Show the 'size' of each of the co72010 events. Size is the total number of students attending each event.**

In [5]:
(ut_event.withColumnRenamed("id", "event")
 .filter(ut_event("modle")==="co72010")
 .join(ut_attends, "event")
 .join(ut_student, (ut_attends("student")===ut_student("id")))
 .groupBy("event")
 .sum("sze")
 .orderBy(col("sum(sze)").desc)
 .showHTML())

event,sum(sze)
co72013.L01,215
co72013.T02,49
co72013.T04,45
co72013.T01,40
co72013.T05,35
co72013.T06,29
co72013.T03,10
co72013.L02,10


## 7.
**For each post-graduate module, show the size of the teaching team. (post graduate modules start with the code co7).**

In [6]:
(ut_teaches
 .join(ut_event.withColumnRenamed("id", "event"), "event")
 .join(ut_modle.filter(ut_modle("id").like("co7%")), 
       (ut_event("modle")===ut_modle("id")))
 .select("id", "staff")
 .distinct()
 .groupBy("id")
 .agg(count("staff"))
 .showHTML())

id,count(staff)
co72003,2
co72011,2
co72018,1
co72017,1
co72026,2
co72016,2
co72033,1
co72010,2
co72006,1
co72021,1


## 8.
**Give the full name of those modules which include events taught for fewer than 10 weeks.**

In [7]:
(ut_modle.withColumnRenamed("name", "modle_name")
 .join(ut_event.withColumnRenamed("id", "event"), 
       (ut_modle("id")===ut_event("modle")))
 .join(ut_occurs, "event")
 .groupBy("event", "modle_name")
 .count()
 .orderBy("modle_name")
 .filter(col("count")<10)
 .select("modle_name")
 .distinct()
 .showHTML())

modle_name
Interactivity and the Internet
Internet Multimedia
Project
Languages and Algorithms


## 9.
**Identify those events which start at the same time as one of the co72010 lectures.**

In [8]:
val t = (ut_event
     .join(ut_occurs, (ut_event("id")===ut_occurs("event")))
     .withColumn("time", concat_ws("", col("week"), col("dow"), col("tod"))))
(t.filter(col("modle") !== "co72010")
 .join(t.filter(col("modle") === "co72010")
       .select("time"), "time")
 .select("id")
 .distinct()
 .orderBy("id")
 .showHTML())

id
co12004.T04
co12004.T05
co12005.T01
co12005.T04
co12006.L03
co12008.L01
co12012.T01
co22005.T02
co22005.T04
co22005.T07


t: DataFrame = [id: string, modle: string ... 8 more fields]

## 10.
**How many members of staff have contact time which is greater than the average?**

In [7]:
val t = (ut_event.join(ut_teaches, (ut_event("id")===ut_teaches("event")))
     .join(ut_occurs, (ut_event("id")===ut_occurs("event")))
     .groupBy("staff")
     .agg(sum("duration").alias("duration")))

(t.filter(col("duration") > t.groupBy().sum("duration").collect()(0)(0).asInstanceOf[Long] /
          ut_teaches.select("staff").distinct().count())
 .groupBy()
 .count()
 .showHTML())

count
19


t: DataFrame = [staff: string, duration: bigint]

In [8]:
spark.stop()